In [2]:
import math

In [3]:
class Perceptron:
    def __init__(self,weights,bias):
        """
        inputs : a vector of inputs
        weights : a vector of weights
        eta : learning rate 
        output : the provided output
        """
        self.weights = weights
        self.bias = bias
        self.n = len(self.weights)
    
    def activation_function(self, x):
        # sigmoid function
        return 1/(1+math.exp(-x))
    
    def Hard_activation_function(self, x):
        return 1 if x > 0 else 0
    
    def predict(self,inputs):
        if len(self.weights)!= self.n:
            raise ValueError("Weights don't match inputs")
        x = sum([self.weights[i]*inputs[i] for i in range(self.n)])-self.bias
        return self.Hard_activation_function(x)

In [4]:
class PerceptronLayer:
    def __init__(self, id, num_perceptrons,weights=None,biases=None):
        """
        num_perceptrons : number of perceptrons in the layer
        id : layer id i.e it's number in the network
        weights : matrix of weights for the perceptron where each row corresponds to a perceptron
        biases : vector of biases for each perceptron
        """
        self.layer_id = id
        self.num_perceptrons = num_perceptrons
        if weights is None:
            weights = [[0.5 for _ in range(num_perceptrons)] for _ in range(num_perceptrons)]
        if biases is None:
            biases = [0.0 for _ in range(num_perceptrons)]
        if len(weights) != num_perceptrons or len(biases) != num_perceptrons:
            raise ValueError("Weights or biases dimensions do not match number of perceptrons")
        self.layer = [Perceptron(weights[i], biases[i]) for i in range(num_perceptrons)] 
    
    def predict(self, inputs):
        output = inputs
        for neuron in self.layer:
            output = [neuron.predict(output)]
        return output

In [5]:
Layer = PerceptronLayer(1,3)
print(Layer.predict([1,0,1])) 

IndexError: list index out of range

In [ ]:
class FeedForwardNeuralNetwork:
    def __init__(self, inputs,outputs,num_Hidden_layers,neurons_per_hidden_layer):
        """
        inputs : number of inputs
        outputs : number of outputs
        num_Hidden_layers : number of hidden layers
        neurons_per_hidden_layer : list of numbers containing number of neurons per hidden layer
        """
        self.inputs = inputs
        self.outputs = outputs
        self.num_Hidden_layers = num_Hidden_layers
        self.neurons_per_hidden_layer = neurons_per_hidden_layer
    
    def predict(self, inputs):
        return self.mlp.predict(inputs)